# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 8 2022 10:02AM,239094,19,ACG-2102490278,ACG North America LLC,Released
1,Jun 8 2022 10:02AM,239094,19,ACG-2102490277,ACG North America LLC,Released
2,Jun 8 2022 10:01AM,239093,15,7913461-BO,"Mizner Bioscience, LLC",Released
3,Jun 8 2022 10:01AM,239093,15,7917968-BO,"Mizner Bioscience, LLC",Released
4,Jun 8 2022 10:01AM,239093,15,7949040-BO,"Mizner Bioscience, LLC",Released
5,Jun 8 2022 10:01AM,239093,15,7979996-BO,"Mizner Bioscience, LLC",Released
6,Jun 8 2022 10:01AM,239093,15,7979998-BO,"Mizner Bioscience, LLC",Released
7,Jun 8 2022 10:01AM,239093,15,7979999-BO,"Mizner Bioscience, LLC",Released
8,Jun 8 2022 10:01AM,239093,15,8010829-BO,"Mizner Bioscience, LLC",Released
9,Jun 8 2022 10:01AM,239093,15,8037192-BO,"Mizner Bioscience, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
56,239089,Released,2
57,239090,Released,1
58,239091,Released,1
59,239093,Released,14
60,239094,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239089,NaN,NaN,2.0
239090,NaN,NaN,1.0
239091,NaN,NaN,1.0
239093,NaN,NaN,14.0
239094,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238942,1.0,14.0,2.0
238984,0.0,2.0,0.0
238989,0.0,1.0,0.0
238992,0.0,0.0,1.0
238994,6.0,52.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238942,1,14,2
238984,0,2,0
238989,0,1,0
238992,0,0,1
238994,6,52,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238942,1,14,2
1,238984,0,2,0
2,238989,0,1,0
3,238992,0,0,1
4,238994,6,52,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238942,1,14,2
1,238984,,2,
2,238989,,1,
3,238992,,,1
4,238994,6,52,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 8 2022 10:02AM,239094,19,ACG North America LLC
2,Jun 8 2022 10:01AM,239093,15,"Mizner Bioscience, LLC"
16,Jun 8 2022 9:56AM,239089,15,"Carwin Pharmaceutical Associates, LLC"
18,Jun 8 2022 9:56AM,239050,15,"Person & Covey, Inc."
30,Jun 8 2022 9:55AM,239091,18,Eye Pharma Inc
31,Jun 8 2022 9:50AM,239090,102,"Senseonics, Incorporated"
32,Jun 8 2022 9:45AM,239049,15,"Mizner Bioscience, LLC"
35,Jun 8 2022 9:44AM,239048,15,"Canton Laboratories, LLC"
36,Jun 8 2022 9:44AM,239047,15,"Alexso, Inc."
37,Jun 8 2022 9:42AM,239045,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 8 2022 10:02AM,239094,19,ACG North America LLC,,,2
1,Jun 8 2022 10:01AM,239093,15,"Mizner Bioscience, LLC",,,14
2,Jun 8 2022 9:56AM,239089,15,"Carwin Pharmaceutical Associates, LLC",,,2
3,Jun 8 2022 9:56AM,239050,15,"Person & Covey, Inc.",,,12
4,Jun 8 2022 9:55AM,239091,18,Eye Pharma Inc,,,1
5,Jun 8 2022 9:50AM,239090,102,"Senseonics, Incorporated",,,1
6,Jun 8 2022 9:45AM,239049,15,"Mizner Bioscience, LLC",,,3
7,Jun 8 2022 9:44AM,239048,15,"Canton Laboratories, LLC",,,1
8,Jun 8 2022 9:44AM,239047,15,"Alexso, Inc.",,,1
9,Jun 8 2022 9:42AM,239045,15,"Brookfield Pharmaceuticals, LLC",,,32


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 10:02AM,239094,19,ACG North America LLC,2,,
1,Jun 8 2022 10:01AM,239093,15,"Mizner Bioscience, LLC",14,,
2,Jun 8 2022 9:56AM,239089,15,"Carwin Pharmaceutical Associates, LLC",2,,
3,Jun 8 2022 9:56AM,239050,15,"Person & Covey, Inc.",12,,
4,Jun 8 2022 9:55AM,239091,18,Eye Pharma Inc,1,,
5,Jun 8 2022 9:50AM,239090,102,"Senseonics, Incorporated",1,,
6,Jun 8 2022 9:45AM,239049,15,"Mizner Bioscience, LLC",3,,
7,Jun 8 2022 9:44AM,239048,15,"Canton Laboratories, LLC",1,,
8,Jun 8 2022 9:44AM,239047,15,"Alexso, Inc.",1,,
9,Jun 8 2022 9:42AM,239045,15,"Brookfield Pharmaceuticals, LLC",32,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 10:02AM,239094,19,ACG North America LLC,2,,
1,Jun 8 2022 10:01AM,239093,15,"Mizner Bioscience, LLC",14,,
2,Jun 8 2022 9:56AM,239089,15,"Carwin Pharmaceutical Associates, LLC",2,,
3,Jun 8 2022 9:56AM,239050,15,"Person & Covey, Inc.",12,,
4,Jun 8 2022 9:55AM,239091,18,Eye Pharma Inc,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 10:02AM,239094,19,ACG North America LLC,2.0,NaN,NaN
1,Jun 8 2022 10:01AM,239093,15,"Mizner Bioscience, LLC",14.0,NaN,NaN
2,Jun 8 2022 9:56AM,239089,15,"Carwin Pharmaceutical Associates, LLC",2.0,NaN,NaN
3,Jun 8 2022 9:56AM,239050,15,"Person & Covey, Inc.",12.0,NaN,NaN
4,Jun 8 2022 9:55AM,239091,18,Eye Pharma Inc,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 10:02AM,239094,19,ACG North America LLC,2.0,0.0,0.0
1,Jun 8 2022 10:01AM,239093,15,"Mizner Bioscience, LLC",14.0,0.0,0.0
2,Jun 8 2022 9:56AM,239089,15,"Carwin Pharmaceutical Associates, LLC",2.0,0.0,0.0
3,Jun 8 2022 9:56AM,239050,15,"Person & Covey, Inc.",12.0,0.0,0.0
4,Jun 8 2022 9:55AM,239091,18,Eye Pharma Inc,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2390530,189.0,37.0,25.0
102,717254,3.0,0.0,0.0
12,1195317,11.0,2.0,0.0
15,1912465,91.0,0.0,0.0
18,478107,2.0,0.0,0.0
19,2151443,46.0,5.0,0.0
20,1434035,55.0,66.0,7.0
21,1673244,7.0,0.0,0.0
22,239061,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2390530,189.0,37.0,25.0
1,102,717254,3.0,0.0,0.0
2,12,1195317,11.0,2.0,0.0
3,15,1912465,91.0,0.0,0.0
4,18,478107,2.0,0.0,0.0
5,19,2151443,46.0,5.0,0.0
6,20,1434035,55.0,66.0,7.0
7,21,1673244,7.0,0.0,0.0
8,22,239061,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,189.0,37.0,25.0
1,102,3.0,0.0,0.0
2,12,11.0,2.0,0.0
3,15,91.0,0.0,0.0
4,18,2.0,0.0,0.0
5,19,46.0,5.0,0.0
6,20,55.0,66.0,7.0
7,21,7.0,0.0,0.0
8,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,189.0
1,102,Released,3.0
2,12,Released,11.0
3,15,Released,91.0
4,18,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,18,19,20,21,22
Status,,,,,,,,,
Completed,25.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0
Executing,37.0,0.0,2.0,0.0,0.0,5.0,66.0,0.0,1.0
Released,189.0,3.0,11.0,91.0,2.0,46.0,55.0,7.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,18,19,20,21,22
0,Completed,25.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0
1,Executing,37.0,0.0,2.0,0.0,0.0,5.0,66.0,0.0,1.0
2,Released,189.0,3.0,11.0,91.0,2.0,46.0,55.0,7.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,18,19,20,21,22
0,Completed,25.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0
1,Executing,37.0,0.0,2.0,0.0,0.0,5.0,66.0,0.0,1.0
2,Released,189.0,3.0,11.0,91.0,2.0,46.0,55.0,7.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()